In [1]:
from threeML import *
import matplotlib.pyplot as plt

"""

This example demonstrates the use of FermiLatLike to link a parameter between two sources in different ROI & time intervals.

In this use case, we will choose two EBL-attenuated GRB point sources and link the 'attenuation' parameter between them.

"""

/home/xavier/miniconda3/envs/threeml_fermi/lib/python2.7/site-packages/threeML/__init__.py:13: UserWarning: No DISPLAY variable set. Using backend for graphics without display (Agg)
  "No DISPLAY variable set. Using backend for graphics without display (Agg)"
/home/xavier/miniconda3/envs/threeml_fermi/lib/python2.7/site-packages/astromodels/core/parameter.py:555: UserWarning: We have set the min_value of xc to 1e-99 because there was a postive transform
  warnings.warn('We have set the min_value of %s to 1e-99 because there was a postive transform' % self.path)

WARNING UserWarning: Using default configuration from /home/xavier/miniconda3/envs/threeml_fermi/lib/python2.7/site-packages/threeML/data/threeML_config.yml. You might want to copy it to /home/xavier/.threeML/threeML_config.yml to customize it and avoid this warning.


WARNING RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88


WARNING RuntimeWarning: numpy.dtype size changed, ma

"\n\nThis example demonstrates the use of FermiLatLike to link a parameter between two sources in different ROI & time intervals.\n\nIn this use case, we will choose two EBL-attenuated GRB point sources and link the 'attenuation' parameter between them.\n\n"

In [2]:
#First, we define a couple helper functions to encapsulate doTimeResolvedLike and create models for our sources.

def doLAT(OUTFILE,RA,DEC,TSTARTS,TSTOPS,ROI=5.0,ZMAX=105,EMIN=65,EMAX=100000,IRF='p8_transient010e', data_path='./'):
        '''
        
        This is a simple wrapper of the doTimeResolvedLike of gtburst
        
        TSTARTS,TSTOPS can be arrays if you want to run multiple intervals
        
        '''
        analysis_dir = '%s_analysis_%s-%s' % (OUTFILE,EMIN,EMAX) 
        os.system('mkdir -p %s' % analysis_dir)
        os.chdir(analysis_dir)
        exe='$CONDA_PREFIX/lib/python2.7/site-packages/fermitools/GtBurst/scripts/doTimeResolvedLike.py'
        #exe='doTimeResolvedLike.py'
        args={}
        args['outfile'] = OUTFILE
        args['ra']      = RA
        args['dec']     = DEC
        args['roi']     = ROI
        TSTARTS_str     = ''
        TSTOPS_str      = ''
        for t0,t1 in zip(TSTARTS,TSTOPS):
            TSTARTS_str+='%s, ' % t0
            TSTOPS_str+='%s, ' % t1
        TSTARTS_str=TSTARTS_str[:-2]
        TSTOPS_str=TSTOPS_str[:-2]
        args['tstarts'] = "'%s'" % TSTARTS_str
        args['tstops']  = "'%s'" % TSTOPS_str
        args['zmax']    = ZMAX
        args['emin']    = EMIN
        args['emax']    = EMAX
        args['irf']     = IRF
        args['galactic_model']   = "'template (fixed norm.)'"
        args['particle_model']   = "'isotr template'"
        args['tsmin']            = 25
        args['strategy']         = 'time'
        args['thetamax']         = 180
        args['spectralfiles']    = 'yes'
        args['liketype']         = 'unbinned'
        args['optimizeposition'] = 'no'
        args['datarepository']   = data_path
        args['flemin']           = 100.
        args['flemax']           = 10000
        args['fgl_mode']         = 'fast'
        triggername              = OUTFILE
        for k,i in args.items():
            exe+=' --%s %s' % (k,i)
        exe+=' %s' % triggername
        print(exe)

        os.system(exe)
    
        return analysis_dir

In [3]:
def createSrcModel(src_name,ra,dec,redshift,index):
    powerlaw = Powerlaw()
    powerlaw.index.prior = Uniform_prior(lower_bound=-5.0, upper_bound=5.0)
    powerlaw.K.prior = Log_uniform_prior(lower_bound=1.0e-20, upper_bound=1e-10)
    powerlaw.piv     = 5.0e+5
    powerlaw.index   = index
    powerlaw.index.free = False
    
    ebl = EBLattenuation()
    #This attenuation parameter is what we want to link across multiple source models.
    ebl.attenuation.prior = Uniform_prior(lower_bound = 0.0, upper_bound = 2.0)
    ebl.attenuation.fix = False
    
    source = powerlaw*ebl
    source.redshift_2 = redshift * u.dimensionless_unscaled
    
    return PointSource(src_name, ra, dec, spectral_shape = source)

In [4]:
#Relevant GRB data, sourced from the GRB catalog
trigger_id = 'bn080916009'
ra, dec, redshift, index, tstart, tstop = 119.889999, -56.700001, 4.350, -2.072603, 3.03, 1531.780029

#Create the source model
source_1 = createSrcModel(trigger_id, ra, dec, redshift, index)

#Calls doTimeResolvedLike helper function, creating a directory from which we can sift the appropriate fit files
doLAT(trigger_id, ra, dec, [tstart], [tstop])

#The files retrieved by the doLAT step:
ft2File = os.path.expandvars('${HOME}/FermiData') + '/%s/gll_ft2_tr_%s_v00.fit'%(trigger_id,trigger_id)
directory= '%s/interval%s-%s/' % ('.', tstart, tstop)
eventFile = glob.glob("%s/*_filt.fit" % directory)[0]
expomap = glob.glob("%s/*_filt_expomap.fit" % directory)[0] 
ltcube = glob.glob("%s/*_filt_ltcube.fit" % directory)[0]

#create LAT plugin using this data -- importantly, passing the source name:
lat_plugin_1 = FermiLATLike(trigger_id, eventFile, ft2File, ltcube, 'unbinned', expomap, source_name = trigger_id)

"""

If you prefer not to restrict the plugin to an individual source, initialize the FermiLatLike without source_name, or use:

lat_plugin_1.clear_source_name()


Now, if we chose to fit just one GRB, we could do so now. For example, using JointLikelihood from classicMLE:

jl = JointLikelihood(Model(source_1),DataList(lat_plugin_1),verbose=True)
jl.set_minimizer('minuit')
jl.fit()
"""

$CONDA_PREFIX/lib/python2.7/site-packages/fermitools/GtBurst/scripts/doTimeResolvedLike.py --tstops '1531.780029' --zmax 105 --roi 5.0 --emax 100000 --flemax 10000 --thetamax 180 --particle_model 'isotr template' --datarepository ./ --galactic_model 'template (fixed norm.)' --strategy time --tsmin 25 --optimizeposition no --ra 119.889999 --fgl_mode fast --irf p8_transient010e --spectralfiles yes --outfile bn080916009 --liketype unbinned --flemin 100.0 --tstarts '3.03' --emin 65 --dec -56.700001 bn080916009


"\n\nIf you prefer not to restrict the plugin to an individual source, initialize the FermiLatLike without source_name, or use:\n\nlat_plugin_1.clear_source_name()\n\n\nNow, if we chose to fit just one GRB, we could do so now. For example, using JointLikelihood from classicMLE:\n\njl = JointLikelihood(Model(source_1),DataList(lat_plugin_1),verbose=True)\njl.set_minimizer('minuit')\njl.fit()\n"

In [5]:
#Now, let's bring in our second source
trigger_id = 'bn090102122'
ra, dec, redshift, index, tstart, tstop = 127.559998, 33.459999, 1.547, -0.062906, 3915.889893, 4404.799805

source_2 = createSrcModel(trigger_id, ra, dec, redshift, index)

doLAT(trigger_id, ra, dec, [tstart], [tstop])

ft2File = os.path.expandvars('${HOME}/FermiData') + '/%s/gll_ft2_tr_%s_v00.fit'%(trigger_id,trigger_id)
directory= '%s/interval%s-%s/' % ('.', tstart, tstop)
eventFile = glob.glob("%s/*_filt.fit" % directory)[0]
expomap = glob.glob("%s/*_filt_expomap.fit" % directory)[0] 
ltcube = glob.glob("%s/*_filt_ltcube.fit" % directory)[0]

lat_plugin_2 = FermiLATLike(trigger_id, eventFile, ft2File, ltcube, 'unbinned', expomap, source_name = trigger_id)

$CONDA_PREFIX/lib/python2.7/site-packages/fermitools/GtBurst/scripts/doTimeResolvedLike.py --tstops '4404.799805' --zmax 105 --roi 5.0 --emax 100000 --flemax 10000 --thetamax 180 --particle_model 'isotr template' --datarepository ./ --galactic_model 'template (fixed norm.)' --strategy time --tsmin 25 --optimizeposition no --ra 127.559998 --fgl_mode fast --irf p8_transient010e --spectralfiles yes --outfile bn090102122 --liketype unbinned --flemin 100.0 --tstarts '3915.889893' --emin 65 --dec 33.459999 bn090102122


In [10]:
#Here, we create the model and link the desired parameters.
model = Model(source_1, source_2)
model.link(model.bn080916009.spectrum.main.composite.attenuation_2, model.bn090102122.spectrum.main.composite.attenuation_2)

#We set the model for each of the plugins.
#This is a second opportunity to change the plugin source's name, if desired, by including a source_name flag.
lat_plugin_1.set_model(model)
lat_plugin_2.set_model(model)
datalist = DataList(lat_plugin_1, lat_plugin_2)

#Let's use ultranest:
bayes = BayesianAnalysis(model, datalist)

for name in ['bn080916009','bn090102122']:
    getattr(bayes.likelihood_model,'%s_GalacticTemplate_Value'%name).set_uninformative_prior(Uniform_prior) 
    getattr(bayes.likelihood_model,'%s_IsotropicTemplate_Normalization'%name).set_uninformative_prior(Uniform_prior)

bayes.set_sampler('ultranest')
bayes.sampler.setup()

bayes.sample(quiet=False)

Setting single point source bn080916009 ... 

Found Isotropic template for irf P8R3_TRANSIENT010E_V2: /home/xavier/miniconda3/envs/threeml_fermi/share/fermitools/refdata/fermi/galdiffuse/iso_P8R3_TRANSIENT010E_V2.txt

Found Galactic template for IRF. P8R3_TRANSIENT010E_V2: /home/xavier/miniconda3/envs/threeml_fermi/share/fermitools/refdata/fermi/galdiffuse/gll_iem_v07.fits

Cutting the template around the ROI: 

Setting single point source bn090102122 ... 

Found Isotropic template for irf P8R3_TRANSIENT010E_V2: /home/xavier/miniconda3/envs/threeml_fermi/share/fermitools/refdata/fermi/galdiffuse/iso_P8R3_TRANSIENT010E_V2.txt

Found Galactic template for IRF. P8R3_TRANSIENT010E_V2: /home/xavier/miniconda3/envs/threeml_fermi/share/fermitools/refdata/fermi/galdiffuse/gll_iem_v07.fits

Cutting the template around the ROI: 

Setting single point source bn080916009 ... 

Found Isotropic template for irf P8R3_TRANSIENT010E_V2: /home/xavier/miniconda3/envs/threeml_fermi/share/fermitools/refdat

INFO:ultranest:Sampling 400 live points from prior ...
DEBUG:ultranest:minimal_widths_sequence: [(-inf, 400.0), (inf, 400.0)]


ImportError: cannot import name GridspecLayout

In [ ]:
#From here, we can print the results, which give us the fitted value for our linked parameter

bayes.results.display()

In [ ]:
#We can also plot the figure these results produce.

fig = plot_spectra(bayes.results, flux_unit = 'erg2/(cm2 s keV)', fit_cmap = 'viridis', contour_cmap = 'viridis', contour_style_kwargs = dict(alpha=0.1), energy_unit = 'MeV', ene_min = 65, ene_max = 100000 )
fig.show()

#For more on Bayesian Analyses, see the bayesian_tutorial notebook.